In [1]:
import os
import torch
from PIL import Image
from io import BytesIO
from rembg.bg import remove
from multiprocessing import Pool, cpu_count
import concurrent.futures
import threading
import time

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

using cuda device


In [3]:
!pip install -U pillow


In [4]:
!pip install rembg

In [5]:
print_lock = threading.Lock()

In [6]:
def remove_background(input_data):
    try:
        return remove(input_data, alpha_matting=True)
    except Exception as e:
        with print_lock:
            print(f"Error during background removal: {e}")
        return None

In [7]:
def process_image(input_path, output_folder, background_img_path):
    try:
        # Load the image
        img = Image.open(input_path)
        original_filename = os.path.basename(input_path)
        
        # Save the original image
        original_output_path = os.path.join(output_folder, 'original', original_filename)
        img.save(original_output_path, format='JPEG')
        
        # Remove the background
        with open(input_path, 'rb') as f:
            input_data = f.read()
        
        subject = remove_background(input_data)
        if subject is None:
            return
        
        # Save the masked image to a BytesIO object
        masked_img = Image.open(BytesIO(subject)).convert("RGBA")
        
        # Save the masked image
        masked_output_path = os.path.join(output_folder, 'masked', original_filename)
        masked_img.save(masked_output_path, format='PNG')
        
        # Load background image
        background_img = Image.open(background_img_path).convert("RGBA")
        
        # Resize background to match the foreground
        background_img = background_img.resize(masked_img.size)
        
        # Composite the foreground on the background
        final_img = Image.alpha_composite(background_img, masked_img)
        final_img = final_img.convert("RGB")  # Convert back to RGB mode
        
        # Save the final image
        processed_output_path = os.path.join(output_folder, 'processed', original_filename)
        final_img.save(processed_output_path, format='JPEG')
        
        with print_lock:
            print(f"Processed {original_filename} with background {os.path.basename(background_img_path)}")

    except Exception as e:
        with print_lock:
            print(f"Error processing {input_path} with {background_img_path}: {e}")

In [8]:
def process_pair(input_folder, output_folder, background_folder, fg_filename, bg_filename):
    fg_path = os.path.join(input_folder, fg_filename)
    bg_path = os.path.join(background_folder, bg_filename)
    process_image(fg_path, output_folder, bg_path)


In [9]:
def process_folder(input_folder, output_folder, background_folder):
    os.makedirs(os.path.join(output_folder, 'original'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'masked'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'processed'), exist_ok=True)
    
    fg_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".jpeg") or f.endswith(".jpg")])
    bg_files = sorted([f for f in os.listdir(background_folder) if f.endswith(".jpeg") or f.endswith(".jpg")])
    
    # Ensure equal number of frames
    if len(fg_files) != len(bg_files):
        raise ValueError("The number of foreground and background frames must be the same.")
    
    # Prepare arguments for multiprocessing
    args = [(input_folder, output_folder, background_folder, fg_files[i], bg_files[i]) for i in range(len(fg_files))]
    
    # Use threading for I/O operations
    with concurrent.futures.ThreadPoolExecutor(max_workers=cpu_count()) as executor:
        executor.map(lambda p: process_pair(*p), args)


In [10]:
input_folder = r"E:\My projects\Rembg\frames_extraction_HD\fg_frames"
output_folder = r"E:\My projects\Rembg\output_frames_rembg"
background_folder = r'E:\My projects\Rembg\frames_extraction_HD\bg_frames'


In [11]:
start_time = time.time()
process_folder(input_folder, output_folder, background_folder)
end_time = time.time()

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shif

In [12]:
print(f"Processing completed in {end_time - start_time} seconds")

Processing completed in 608.9995839595795 seconds
